In [1]:
import pickle

import numpy as np
import pandas as pd
from catboost import CatBoostClassifier

Я вижу 2 способа решения этой задачи:
1. Взять наиболее значимые признаки вычисленные при помощи бутстрапа или статистических критериев других и в 3 экспериментировать с этими колонками
2. Взять уже обученный на всём трейне катбуст и взять из него 3 самых значимых столбца и взять из них границы и использовать их как правило антифрода

In [2]:
# Решил делать с катбустом
with open("../src/app/core/catboost_model.pkl", "rb") as f:
    model = pickle.load(f)

In [3]:
# Загрузим трейн датасет, полученный в hw4
data = pd.read_csv('../src/app/core/train_X.csv')

In [4]:
feature_importance = pd.DataFrame(model.get_feature_importance(), columns=['importance'])
border = pd.DataFrame(model.get_borders().values())
feature_names = list(data.columns)
feature_names.remove('SK_ID_CURR')
feature_names.remove('TARGET')
feature_names = pd.DataFrame(feature_names, columns=['name'])

In [5]:
# Объединим таблицы и отсортируем их по важности фичи
data_for_rules = pd.concat([feature_names, feature_importance, border], axis=1).sort_values(
    'importance', ascending=False).reset_index(drop=True)
data_for_rules.head(3)

,name,importance,0,1,2,3,4,5,6,7,...,46,47,48,49,50,51,52,53,54,55
0,WEIGHTED_EXT_SOURCE,6.434536,-3.402823e+38,0.120988,0.129951,0.168538,0.174112,0.190508,0.200611,0.205222,...,0.594225,0.642128,0.666649,0.696929,0.699536,0.736625,0.741814,NaN,NaN,NaN
1,INTEREST_RATE_v1,3.736829,2.923760e-02,0.029240,0.029330,0.029340,0.029664,0.030572,0.031496,0.032255,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,DEBT_RATIO,3.036217,1.262867e-01,0.498487,0.517171,0.539629,0.541906,0.580905,0.587385,0.606625,...,0.956937,0.975938,0.986005,0.992775,0.996735,1.001551,1.009983,1.068054,1.313749,1.591746


In [6]:
def check_target(testing_data: pd.DataFrame) -> float:
    '''Функция выдающая процент TARGET в выборке'''
    return testing_data['TARGET'].value_counts()[1] / len(testing_data)

In [7]:
# Выведем процент фрода во всём датасете
print(f'{check_target(data)}%')

0.08486411000107423%


In [8]:
from typing import Tuple
from operator import lt, le, ge, gt


def get_best_rule(rule: pd.Series, data_for_testing: pd.DataFrame) -> Tuple[str, str, float, float]:
    """Функция берёт правило и датасет и ищет наилучшее правило"""
    rule = pd.DataFrame(rule)
    # <,<=,>=,>
    operators = [lt, le, ge, gt]
    best_operator = None
    best_value = None
    best_result = 0
    for operator in operators:
        for i in range(len(rule) - 3):
            if best_operator is None and best_value is None:
                best_operator = operator
                best_value = rule.loc[i]
            if np.isnan(float(rule.loc[i])):
                break
            index = pd.Series(np.array(operator(data_for_testing[rule.loc['name']],
                                                float(rule.loc[i]))
                                       )[:, 0])

            test_data = data_for_testing.loc[index]
            if len(test_data) != 0:
                result = check_target(test_data)
                # Так как нельзя больше 5% данных брать
                if index.value_counts()[1] / len(index) < 0.05:
                    if result > best_result:
                        best_operator = operator
                        best_value = rule.loc[i]
                        best_result = result

    return np.array(rule.loc['name'])[0], best_operator, float(best_value), best_result


# Получим все лучшие правила
name, operation, value, result = get_best_rule(data_for_rules.iloc[0], data)
print(f'{name} {operation.__name__} {value} with result {result}%')

name2, operation2, value2, result2 = get_best_rule(data_for_rules.iloc[1], data)
print(f'{name2} {operation2.__name__} {value2} with result {result2}%')

name3, operation3, value3, result3 = get_best_rule(data_for_rules.iloc[2], data)
print(f'{name3} {operation3.__name__} {value3} with result {result3}%')

WEIGHTED_EXT_SOURCE lt 0.12098759412765503 with result 0.2600297176820208%
INTEREST_RATE_v1 ge 0.10916344821453094 with result 0.08717948717948718%
DEBT_RATIO ge 1.313748836517334 with result 0.17358490566037735%


Как мы видим правила антифрода убирают большой процент фрода
1. 26% фрода при WEIGHTED_EXT_SOURCE < 0.12098759412765503
2. 8.7% фрода при INTEREST_RATE_v1 >= 0.10916344821453094
3. 17.3% фрода при DEBT_RATIO >= 1.313748836517334

В теории можно перебрать все столбцы и их значения из обученного катбуста, но я не стал, так как это вычислительно сложно и будет долго считать

In [9]:
# Надо чтобы равномерно было в каждом датасете таргетов
from sklearn.model_selection import StratifiedKFold

my_cv = StratifiedKFold(n_splits=5)

folds = pd.DataFrame(my_cv.split(data, np.array(data['TARGET']).ravel()))[1]
train_indicies = np.concatenate([folds[0], folds[1], folds[2], folds[3]])
val_indicies = folds[4]

train = data.iloc[train_indicies].reset_index(drop=True)
validation = data.iloc[val_indicies].reset_index(drop=True)

In [10]:
# Применим правила
def apply_rules(data_for_rules: pd.DataFrame, names: list, operations: list,
                values: list) -> pd.DataFrame:
    assert len(names) == len(operations)
    assert len(operations) == len(values)
    for i in range(len(names)):
        if i == 0:
            my_operation = operations[i]
            index = np.array(my_operation(data_for_rules[names[i]],
                                          float(values[i]))
                             )
            # Инвертируем индекс, чтобы не брать фродовые случаи
            index = ~index
        else:
            my_operation = operations[i]
            index = np.logical_and(index, ~np.array(my_operation(data_for_rules[names[i]],
                                                                 float(values[i]))))
    return data_for_rules[pd.Series(index)].reset_index(drop=True)


train_after_rule = apply_rules(train,
                               [name, name2, name3],
                               [operation, operation2, operation3],
                               [value, value2, value3])

validation_after_rule = apply_rules(validation,
                                    [name, name2, name3],
                                    [operation, operation2, operation3],
                                    [value, value2, value3])

In [16]:
from sklearn.metrics import roc_auc_score

ctbst = CatBoostClassifier(depth=8,
                           l2_leaf_reg=10,
                           silent=True,
                           border_count=254)
feature_names = list(data.columns)
feature_names.remove('SK_ID_CURR')
feature_names.remove('TARGET')

train_Y = np.array(train_after_rule['TARGET'])
train_X = train_after_rule[feature_names]

val_Y = np.array(validation_after_rule['TARGET'])
val_X = validation_after_rule[feature_names]

ctbst.fit(train_X, train_Y)
preds = ctbst.predict_proba(val_X)[:, 1]
print(roc_auc_score(val_Y, preds))

0.7496935268533778
0.7496935268533778


roc_auc стал 0.7496 - это лучше, чем без правил антифрода, без правил модель предсказывала roc_auc_score=0.7491

Правила сформированы более менее нормально и выделяют больший процент фрода, чем есть в процент фрода в датасете